<h2 style='color: crimson; border-bottom: 1px seagreen dotted'>MUMBAI TRAVEL TIMES {Q4 2019}</h2>

Access Dataset here ---> [Uber Movement Mumbai](https://movement.uber.com/explore/mumbai/travel-times/query?si=254&ti=&ag=hexclusters&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2020-03-01&dt[dr][ed]=2020-03-31&cd=&sa;=72.9132679,19.1334302&sdn=Indian%20Institute%20of%20Technology%20Bombay,%20Main%20Gate%20Rd,%20IIT%20Area,%20Powai,%20Mumbai,%20Maharashtra&lng.=72.8508724&lat.=19.122537&z.=10.01&lang=en-US)

![Mumbai Trave Times](mumbai_hexagons.png)
<p style='color:orange; padding-left: 20em; border: seagreen 1px dotted'> MUMBAI ORIGIN DESTINATION TRAVELTIME HEATMAP </p>

In [57]:
# install requirements
!pip install -q -r req.txt

In [2]:
# Unzip if not already done
import os
if not 'mumbai-hexclusters-2019-4-All-HourlyAggregate.csv' in os.listdir():
    !unzip mumbai-hexclusters-2019-4-All-HourlyAggregate.zip

In [59]:
# All imports


from prettyprinter import pprint 

import numpy as np
import pandas as pd
import json


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import seaborn as sns


import tensorly as tl


In [4]:
# read dataframe and metadata

df = pd.read_csv('mumbai-hexclusters-2019-4-All-HourlyAggregate.csv')
hexagons = json.load(open('mumbai_hexclusters.json'))['features']

In [5]:
# peek into dataframe
df.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,171,206,15,2949.31,1139.79,2772.51,1.40
1,255,439,10,5045.33,839.06,4978.27,1.18
2,110,18,0,2932.71,537.47,2886.25,1.19
3,38,7,5,2690.29,741.75,2591.41,1.31
4,277,219,10,4848.73,730.67,4796.48,1.16


```python

### definition ###
hod = hour of day
```


In [6]:
# peek metadata
pprint(hexagons[0])

{
    'type': 'Feature',
    'properties': {
        'MOVEMENT_ID': '1',
        'DISPLAY_NAME': '0, Beverly Park Rd, Chandan Shanti, Mira Road East'
    },
    'geometry': {
        'type': 'Polygon',
        'coordinates': [
            [
                [72.87164792379636, 19.28533060455805],
                [72.8710709563304, 19.28699016299829],
                [72.87222988623245, 19.288411970470293],
                [72.87165291120459, 19.290071498239048],
                [72.86991701541635, 19.29030919072361],
                [72.86934004196853, 19.29196866000375],
                [72.86760418931208, 19.292206301216417],
                [72.86702721744616, 19.293865712003942],
                [72.86529140792581, 19.294103301944087],
                [72.86413260267913, 19.292681516129214],
                [72.86470956539794, 19.291022133151962],
                [72.86355078499479, 19.289600351689437],
                [72.86412774015048, 19.287940938034637],
                [72.862

In [25]:
#test all unique
gdf = df.groupby(['sourceid','dstid', 'hod']).count()

if np.all(gdf==1): print('All good, no repeats')
else: print('Some src-dst-hod are repeated!!!')

All good, no repeats


In [83]:
origins = sorted(df.sourceid.unique())
destinations = sorted(df.dstid.unique())
hods = sorted(df.hod.unique())

origin_max = max(origins)
dest_max = max(destinations)
hod_max = max(hods)

In [92]:
# Make numpy tensor
nptensor = np.zeros(     (   dest_max+1, origin_max+1, hod_max+1  )      ) 
        
def transfer(x):
    nptensor[x.dstid ,x.sourceid, x.hod] = x.mean_travel_time
    return 0

df.pipe(transfer)

0

In [112]:
nptensor[300,3,10]

3691.71

In [113]:
df[df.sourceid==3] [ df.dstid==300 ] [ df.hod==10]

<ipython-input-113-481afcb8536b>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.sourceid==3] [ df.dstid==300 ] [ df.hod==10]
<ipython-input-113-481afcb8536b>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.sourceid==3] [ df.dstid==300 ] [ df.hod==10]


,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
2486014,3,300,10,3691.71,764.25,3626.25,1.2
